# Processing Big Data - Data Ingestion
© Explore Data Science Academy

## Honour Code
I **PRECIOUS**, **AROVO**, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the [EDSA honour code](https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).
    Non-compliance with the honour code constitutes a material breach of contract.



## Context 

To work constructively with any dataset, one needs to create an ingestion profile to make sure that the data at the source can be readily consumed. For this section of the predict, as the Data Engineer in the team, you will be required to design and implement the ingestion process. For the purposes of the project the AWS cloud storage service, namely, the S3 bucket service will act as your data source. All the data required can be found [here](https://processing-big-data-predict-stocks-data.s3.eu-west-1.amazonaws.com/stocks.zip).

<div align="center" style="width: 600px; font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/data_engineering/transform/predict/DataIngestion.jpg"
     alt="Data Ingestion"
     style="float: center; padding-bottom=0.5em"
     width=40%/>
     <p><em>Figure 1. Data Ingestion</em></p>
</div>

Your manager, Gnissecorp Atadgib, knowing very well that you've recently completed your Data Engineering qualification, asks you to make use of Apache Spark for the ingestion as well as the rest of the project. His rationale being, that stock market data is generated every day and is quite time-sensitive and would require scalability when deploying to a production environment. 

## Dataset - US Nasdaq




<div align="center" style="width: 600px; font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/data_engineering/transform/predict/Nasdaq.png"
     alt="Nasdaq"
     style="float: center; padding-bottom=0.5em"
     width=50%/>
     <p><em>Figure 2. Nasdaq</em></p>
</div>

The data that you will be working with is a historical snapshot of market data taken from the Nasdaq electronic market. This dataset contains historical daily prices for all tickers currently trading on Nasdaq. The up-to-date list can be found on their [website](https://www.nasdaq.com/)


The provided data contains price data dating back from 02 January 1962 up until 01 April 2020. The data found in the S3 bucket has been stored in the following structure:

```
     stocks/<Year>/<Month>/<Day>/stocks.csv
```
Each CSV file for every trading day contains the following details:
- **Date** - specifies trading date
- **Open** - opening price
- **High** - maximum price during the day
- **Low** - minimum price during the day
- **Close** - close price adjusted for splits
- **Adj Close** - close price adjusted for both dividends and splits
- **Volume** - the number of shares that changed hands during a given day

## Basic initialisation
To get you started, let's import some basic Python libraries as well as Spark modules and functions.

In [1]:
import pandas as pd
import numpy as np
import pyspark
import matplotlib.pyplot as plt

from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructField, StructType, IntegerType, FloatType, StringType, TimestampType

Remember that we need a `SparkContext` and `SparkSession` to interface with Spark.
We will mostly be using the `SparkContext` to interact with RDDs and the `SparkSession` to interface with Python objects.

> ℹ️ **Instructions** ℹ️
>
>Initialise a new **Spark Context** and **Session** that you will use to interface with Spark.

In [2]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable

os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [3]:
# Defining SparkContext and SparkSession

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

## Investigate dataset schema
At this point, it is enough to read in a single file to ascertain the data structure. You will be required to use the information obtained from the small subset to create a data schema. This data schema will be used when reading the entire dataset using Spark.

> ℹ️ **Instructions** ℹ️
>
>Make use of Pandas to read in a single file and investigate the plausible data types to be used when creating a Spark data schema. 
>
>*You may use as many coding cells as necessary.*

In [4]:
single_file_path = r"C:\Users\USER\Desktop\ExploreDE\processing-big-data-predict\stocks\1962\01\02\stocks.csv"
df = pd.read_csv(single_file_path)
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,stock
0,1962-01-02,6.532155,6.556185,6.532155,6.532155,1.536658,55900.0,AA
1,1962-01-02,6.125844,6.160982,6.125844,6.125844,1.414651,59700.0,ARNC
2,1962-01-02,0.837449,0.837449,0.823045,0.823045,0.145748,352200.0,BA
3,1962-01-02,1.604167,1.619792,1.588542,1.604167,0.136957,163200.0,CAT
4,1962-01-02,0.000000,3.296131,3.244048,3.296131,0.051993,105600.0,CVX


In [5]:
df.shape

(20, 8)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       20 non-null     object 
 1   Open       20 non-null     float64
 2   High       20 non-null     float64
 3   Low        20 non-null     float64
 4   Close      20 non-null     float64
 5   Adj Close  20 non-null     float64
 6   Volume     20 non-null     float64
 7   stock      20 non-null     object 
dtypes: float64(6), object(2)
memory usage: 1.4+ KB


## Read CSV files

When working with big data, it is often not tenable to keep processing an entire data batch when you are in the process of development - this can be quite time-consuming. If the data is uniform, it is sufficient to work with a smaller subset to create basic functionality. Your manager has identified the year **1962** to perform the initial testing for data ingestion. 

> ℹ️ **Instructions** ℹ️
>
>Read in the data for **1962** using a data schema that purely uses string data types. You will be required to convert to the appropriate data types at a later stage.
>
>*You may use as many coding cells as necessary.*

In [7]:
# Stock data path
path = r"C:\Users\USER\Desktop\ExploreDE\processing-big-data-predict\stocks"

# Year to read the data (.csv files) from.
year = '1962'

# Empty list to store the paths of the CSV list
csv_paths = []

# Looping through the months and days of the year 1962
for month in range(1, 13):
    for day in range(1, 32):
        csv_path = os.path.join(path, year, str(month).zfill(2), str(day).zfill(2), "stocks.csv")
        if os.path.exists(csv_path):
            csv_paths.append(csv_path)

raw_data =spark.read.csv(csv_paths, header=True)

In [8]:
raw_data.show(5)

+----------+------------------+------------------+------------------+------------------+------------------+--------+-----+
|      Date|              Open|              High|               Low|             Close|         Adj Close|  Volume|stock|
+----------+------------------+------------------+------------------+------------------+------------------+--------+-----+
|1962-02-19| 5.839290142059326| 5.907374858856201| 5.839290142059326| 5.863319873809815|1.3863292932510376| 29900.0|   AA|
|1962-02-19| 5.481634140014648|5.5284857749938965| 5.481634140014648| 5.516772747039795|1.2804527282714844| 32000.0| ARNC|
|1962-02-19|0.9074074029922484|0.9156378507614136|0.8991769552230835| 0.903292179107666|0.1614154428243637|619400.0|   BA|
|1962-02-19|1.6770833730697632|1.6927083730697632|1.6614583730697632|1.6770833730697632|0.1440587043762207|170400.0|  CAT|
|1962-02-19|               0.0|3.5788691043853764|              20.0| 3.549107074737549|0.0565012246370315|273600.0|  CVX|
+----------+----

In [9]:
raw_data.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Adj Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- stock: string (nullable = true)



In [10]:
raw_data.describe().show(10, False)

+-------+----------+------------------+------------------+------------------------+--------------------+-----------------------+-----------------+-----+
|summary|Date      |Open              |High              |Low                     |Close               |Adj Close              |Volume           |stock|
+-------+----------+------------------+------------------+------------------------+--------------------+-----------------------+-----------------+-----+
|count  |5106      |5106              |5106              |5084                    |5106                |5106                   |5085             |5106 |
|mean   |NULL      |1.0904873526012002|16.757624946793637|15.728619917198033      |16.64199179044607   |5.986642513535306      |540930.2458210423|NULL |
|stddev |NULL      |2.3644535253049095|53.91407348193544 |51.322922898144         |53.546771399008016  |24.64637054715319      |864596.2440525509|NULL |
|min    |1962-01-02|0.0               |0.0               |-0.040435523172560696F44

## Update column names
To make the data easier to work with, you will need to make a few changes:
1. Column headers should all be in lowercase; and
2. Whitespaces should be replaced with underscores.


> ℹ️ **Instructions** ℹ️
>
>Make sure that the column headers are all in lowercase and that any whitespaces are replaced with underscores.
>
>*You may use as many coding cells as necessary.*

In [11]:
raw_data.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'stock']

In [12]:
# Looping over each column in the dataframe to convert all column header to lowercase
# and replace white space with underscore.
for column in raw_data.columns:
    raw_data = raw_data.withColumnRenamed(column, '_'.join(column.split()).lower())
    

In [13]:
# Checking change
raw_data.columns

['date', 'open', 'high', 'low', 'close', 'adj_close', 'volume', 'stock']

In [14]:
raw_data.show(5)

+----------+------------------+------------------+------------------+------------------+------------------+--------+-----+
|      date|              open|              high|               low|             close|         adj_close|  volume|stock|
+----------+------------------+------------------+------------------+------------------+------------------+--------+-----+
|1962-02-19| 5.839290142059326| 5.907374858856201| 5.839290142059326| 5.863319873809815|1.3863292932510376| 29900.0|   AA|
|1962-02-19| 5.481634140014648|5.5284857749938965| 5.481634140014648| 5.516772747039795|1.2804527282714844| 32000.0| ARNC|
|1962-02-19|0.9074074029922484|0.9156378507614136|0.8991769552230835| 0.903292179107666|0.1614154428243637|619400.0|   BA|
|1962-02-19|1.6770833730697632|1.6927083730697632|1.6614583730697632|1.6770833730697632|0.1440587043762207|170400.0|  CAT|
|1962-02-19|               0.0|3.5788691043853764|              20.0| 3.549107074737549|0.0565012246370315|273600.0|  CVX|
+----------+----

## Null Values
Null values often represent missing pieces of data. It is always good to know where your null values lie - so you can quickly identify and remedy any issues stemming from these.

> ℹ️ **Instructions** ℹ️
>
>Write code to count the number of null values found in each column.
>
>*You may use as many coding cells as necessary.*

In [15]:
# Creating dictionary to hold null values
null_counts = {}

# Looping through the raw data columns
for col in raw_data.columns:
    null_count = raw_data.select(F.count(F.when(F.isnull(col), col)).alias(col)).collect()
    null_counts[col] = null_count[0][0]

In [16]:
null_counts

{'date': 0,
 'open': 0,
 'high': 0,
 'low': 22,
 'close': 0,
 'adj_close': 0,
 'volume': 21,
 'stock': 0}

## Data type conversion - The final data schema

Now that we have identified the number of missing values in the data set, we'll move on to convert our data schema to the required data types. 

> ℹ️ **Instructions** ℹ️
>
>Use typecasting to convert the string data types in your current data schema to more appropriate data types.
>
>*You may use as many coding cells as necessary.*

In [17]:
raw_data.printSchema()

root
 |-- date: string (nullable = true)
 |-- open: string (nullable = true)
 |-- high: string (nullable = true)
 |-- low: string (nullable = true)
 |-- close: string (nullable = true)
 |-- adj_close: string (nullable = true)
 |-- volume: string (nullable = true)
 |-- stock: string (nullable = true)



In [18]:
typed_data = raw_data

In [19]:
# Typecasting to convert the string datatypes in data to their appropriate datatype

schema = StructType([
    StructField('date', TimestampType(), True),
    StructField('open', FloatType(), True),
    StructField('high', FloatType(), True),
    StructField('low', FloatType(), True),
    StructField('close', FloatType(), True),
    StructField('adj_close', FloatType(), True),
    StructField('volume', FloatType(), True),
    StructField('stock', StringType(), True),
])

In [20]:
typed_data = spark.read.csv(csv_paths, schema=schema, header=True)

In [21]:
typed_data.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- open: float (nullable = true)
 |-- high: float (nullable = true)
 |-- low: float (nullable = true)
 |-- close: float (nullable = true)
 |-- adj_close: float (nullable = true)
 |-- volume: float (nullable = true)
 |-- stock: string (nullable = true)



In [22]:
typed_data.show(5)

+-------------------+---------+----------+----------+---------+-----------+--------+-----+
|               date|     open|      high|       low|    close|  adj_close|  volume|stock|
+-------------------+---------+----------+----------+---------+-----------+--------+-----+
|1962-02-19 00:00:00|  5.83929|  5.907375|   5.83929|  5.86332|  1.3863293| 29900.0|   AA|
|1962-02-19 00:00:00| 5.481634|  5.528486|  5.481634|5.5167727|  1.2804527| 32000.0| ARNC|
|1962-02-19 00:00:00|0.9074074|0.91563785|0.89917696|0.9032922| 0.16141544|619400.0|   BA|
|1962-02-19 00:00:00|1.6770834| 1.6927084| 1.6614584|1.6770834|  0.1440587|170400.0|  CAT|
|1962-02-19 00:00:00|      0.0|  3.578869|      20.0| 3.549107|0.056501225|273600.0|  CVX|
+-------------------+---------+----------+----------+---------+-----------+--------+-----+
only showing top 5 rows



In [23]:
typed_data.filter(F.col('low').isNull()).count()

42

## Consolidate missing values
We have to check if the data type conversion above was done correctly.
If the casting was not successful, a null value gets inserted into the dataframe. You can thus check for successful conversion by determining if any null values are included in the resulting dataframe.


> ℹ️ **Instructions** ℹ️
>
>Write code to compare the number of invalid entries (nulls) pre-conversion and post-conversion.
>
>*You may use as many coding cells as necessary.*

In [24]:
raw_data.printSchema()

root
 |-- date: string (nullable = true)
 |-- open: string (nullable = true)
 |-- high: string (nullable = true)
 |-- low: string (nullable = true)
 |-- close: string (nullable = true)
 |-- adj_close: string (nullable = true)
 |-- volume: string (nullable = true)
 |-- stock: string (nullable = true)



In [25]:
# List of float data types
price_float = ['open', 'high', 'low', 'close', 'adj_close', 'volume']

In [26]:
# Creating dictionary to hold null values
cons_null_counts = {}

# Copying a copy of the raw_data
test_data = raw_data

# Looping through the columns in the float data types
for col in price_float:
    # Counting the null values pre-casting
    pre_null_count = test_data.select(F.count(F.when(F.isnull(col), col)).alias(col)).collect()
    
    # Cast to FloatType, creating a new column using the withColumn() method.
    # We use the cast() method to cast and use the FloatType defined in Spark.    
    test_data = test_data.withColumn(f'test_type_{col}', F.col(col).cast(FloatType()))
    
    # Counting the null values post-casting
    post_null_count = test_data.select(F.count(F.when(F.isnull(f'test_type_{col}'), f'test_type_{col}')).alias(f'test_type_{col}')).collect()
    
    # Get the difference pre-casting and post-casting
    cons_null_counts[col] = post_null_count[0][0] - pre_null_count[0][0]

In [27]:
cons_null_counts

{'open': 0, 'high': 0, 'low': 20, 'close': 0, 'adj_close': 21, 'volume': 0}

In [28]:
test_data.printSchema()

root
 |-- date: string (nullable = true)
 |-- open: string (nullable = true)
 |-- high: string (nullable = true)
 |-- low: string (nullable = true)
 |-- close: string (nullable = true)
 |-- adj_close: string (nullable = true)
 |-- volume: string (nullable = true)
 |-- stock: string (nullable = true)
 |-- test_type_open: float (nullable = true)
 |-- test_type_high: float (nullable = true)
 |-- test_type_low: float (nullable = true)
 |-- test_type_close: float (nullable = true)
 |-- test_type_adj_close: float (nullable = true)
 |-- test_type_volume: float (nullable = true)



Here you should be able to see if any of your casts went wrong. 
Do not attempt to correct any missing values at this point. This will be dealt with in later sections of the predict.

## Generate parquet files
When writing in Spark, we typically use parquet format. This format allows parallel writing using Spark's optimisation while maintaining other useful things like metadata.

When writing, it is good to make sure that the data is sufficiently partitioned. 

Generally, data should be partitioned with one partition for every 200MB of data, but this also depends on the size of your cluster and executors. 


### Check the size of the dataframe before partitioning

In [29]:
from pyspark.serializers import PickleSerializer, AutoBatchedSerializer

In [30]:
rdd = typed_data.rdd._reserialize(AutoBatchedSerializer(PickleSerializer()))
obj = rdd.ctx._jvm.org.apache.spark.mllib.api.python.SerDe.pythonToJava(rdd._jrdd, True)
size = sc._jvm.org.apache.spark.util.SizeEstimator.estimate(obj)
size_MB = size/1000000
partitions = max(int(size_MB/200), 2)
print(f'The dataframe is {size_MB} MB')

The dataframe is 36.130736 MB


### Write parquet files to the local directory
> ℹ️ **Instructions** ℹ️
>
> Use the **coalesce** function and the number of **partitions** derived above to write parquet files to your local directory 
>
>*You may use as many coding cells as necessary.*

In [32]:
# Writing parquet files to local directory
typed_data.write.parquet('./stock_data_1962')